<a href="https://colab.research.google.com/github/antounes/practical-ml-for-computer-vision/blob/main/02b_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flowers Image Classification Using a Neural Network

## Enable GPU and set up helper functions 

In [ ]:
import tensorflow as tf
print(tf.version.VERSION)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found - On for CPU time!')
else:
  print('Found GPU at: {}'.format(device_name))

In [ ]:
# Helper functions

def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel('Epochs')
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_'+metric]);
    ax[idx].legend(['train_'+metric, 'val_'+metric])

# Call model.predict() on a few images in the evaluation dataset

def plot_predicitons(filename):
  f, ax = plt.subplots(3, 5, figsize=(25, 15))
  dataset = tf.data.TextLineDataset(filename).map(decode_csv)
  for idx, (img, label) in enumerate(dataset.take(15)):
    ax[idx//5, idx%5].imshow(img.numpy());
    batch_image = tf.reshape(img, [1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    batch_pred = model.predict(batch_image)
    pred = batch_pred[0]
    label = CLASS_NAMES(label.numpy())
    pred_label_index = tf.math.argmax(pred).numpy()
    pred_label = CLASS_NAMES[pred_label_index]
    prob = pred[pred_label_index]
    ax[idx//5, idx%5].set_title('{}: {} ({:.4f})'.format(label, pred_label, prob))

def show_trained_weights(model):
  # CLASS_NAMES is ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
  LAYER = 1 # Layer 0 flattens the image, layer=1 is the first dense layer
  WEIGHT_TYPE = 0 # 0 for weight, 1 for bias

  f, ax = plt.subplots(1, 5, figsize=(15, 15))
  for flower in range(len(CLASS_NAMES)):
    weights = model.layers[LAYER].get_weights()[WEIGHT_TYPE][:, flower]
    min_wt = tf.math.reduce_min(weights).numpy()
    max_wt = tf.math.reduce_max(weights).numpy()
    flower_name = CLASS_NAMES[flower]
    print('Scaling weights for {} in {} to {}'.format(flower_name, min_wt, max_wt))
    weights = (weights - min_wt) / (max_wt - min_wt)
    ax[flower].ishow(weights.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS));
    ax[flower].set_title(flower_name);

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0, 1] range
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size
  return tf.image.resize(img, reshape_dims)

CLASS_NAMES = [item.numpy().decode('utf8') 
for item in tf.strings.regex_replace(tf.io.gfile.glob('gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/*'), 
                                     'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/', '')
]
CLASS_NAMES = [item for item in CLASS_NAMES if item.find('.')==-1]
print('These are the available classes:', CLASS_NAMES)

# The label is the index into CLASS_NAMES array
def decode_csv(csv_row):
  record_defaults = ['path', 'flower']
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

## A Neural Network Model

### Training the neural network

In [ ]:
# Adding one non-linear layer in the middle

BATCH_SIZE = 32

train_dataset = tf.data.TextLineDataset(
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/train_set.csv'
).map(decode_csv).batch(BATCH_SIZE)

eval_dataset = tf.data.TextLineDataset(
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/eval_set.csv'
).map(decode_csv).batch(BATCH_SIZE)

# Neural network with one hidden layer
model = tf.keras.Sequential([
          tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
          tf.keras.layers.Dense(units=128, activation='relu'),
          tf.keras.layers.Dense(units=len(CLASS_NAMES), activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)

In [ ]:
training_plot(['loss', 'accuracy'], history)

### Learning rate

In [ ]:
# parametrise to the values in the previous cell

def train_and_evaluate(batch_size=32,
                       lrate=0.001, # default learning rate in Adam constructor
                       l1=0,
                       l2=0,
                       num_hidden=128):
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)

  train_dataset = tf.data.TextLineDataset(
      'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/train_set.csv'
  ).map(decode_csv).batch(batch_size)

  eval_dataset = tf.data.TextLineDataset(
      'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/eval_set.csv'
  ).map(decode_csv).batch(batch_size)

  # Neural network with one hidden layer
  model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
            tf.keras.layers.Dense(units=num_hidden, kernel_regularizer=regularizer, activation='relu'),
            tf.keras.layers.Dense(units=len(CLASS_NAMES),
                                  kernel_regularizer=regularizer,
                                  activation='softmax')
  ])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
  
  history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)
  training_plot(['loss', 'accuracy'], history)
  
  return model

In [ ]:
model = train_and_evaluate(batch_size=32, lrate=0.0001, l1=0, l2=0, num_hidden=128)

In [ ]:
model = train_and_evaluate(batch_size=32, lrate=0.0001, l1=0, l2=0, num_hidden=256)

### Regularisation

In [ ]:
model = train_and_evaluate(batch_size=32, lrate=0.0001, l1=0, l2=0.001, num_hidden=128)

### Early stopping

In [ ]:
regularizer = tf.keras.regularizers.l1_l2(0, 0.001)
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
  tf.keras.layers.Dense(units=128, kernel_regularizer=regularizer, activation='relu'),
  tf.keras.layers.Dense(len(CLASS_NAMES), kernel_regularizer=regularizer, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])

history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10, callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

## Hyperparameter tuning

In [ ]:
!pip install -q keras-tuner

In [ ]:
import kerastuner as kt

def build_model(hp):
  lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
  l1 = 0
  l2 = hp.Choice('l2', values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
  num_hidden = hp.Int('num_hidden', 32, 256, 32)
  
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)

  # Neural network with one hidden layer
  model = tf.keras.Sequential([
        tf.keras.layers.Flatten(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        tf.keras.layers.Dense(
            units=num_hidden,
            kernel_regularizer=regularizer,
            activation='relu'),
        tf.keras.layers.Dense(
            units=len(CLASS_NAMES), 
            kernel_regularizer=regularizer,
            activation='softmax')
                              
  ])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
  return model

batch_size = 32

train_dataset = tf.data.TextLineDataset(
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/train_set.csv'
).map(decode_csv).batch(batch_size)

eval_dataset = tf.data.TextLineDataset(
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/eval_set.csv'
).map(decode_csv).batch(batch_size)

tuner = kt.BayesianOptimization(
    build_model,
    objective=kt.Objective('val_accuracy', 'max'),
    max_trials=10,
    num_initial_points=2,
    overwrite=False) # True to start afresh

tuner.search(
    train_dataset, validation_data=eval_dataset,
    epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        patience=1
    )]
)

In [ ]:
# Get topN trials
topN = 2
for x in range(topN):
  print(tuner.get_best_hyperparameters(topN)[x].values)
  print(tuner.get_best_models(topN)[x].summary())

### Building a DNN

In [ ]:
def train_and_evaluate(batch_size=32, lrate=0.0001, l1=0, l2=0.001, num_hidden=[64, 16]):
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)

  train_dataset = tf.data.TextLineDataset(
      'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/train_set.csv'
  ).map(decode_csv).batch(batch_size)

  eval_dataset = tf.data.TextLineDataset(
      'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/eval_set.csv'
  ).map(decode_csv).batch(32) # Batch size for eval dataset doesn't matter

  # NN with multiple hidden layers
  layers = [
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), 
                            name='input_pixels'),
  ]
  layers.append([
    tf.keras.layers.Dense(nodes,
                          kernel_regularizer=regularizer,
                          activation='relu',
                          name=f'hidden_dense_{hno}')
  for hno, nodes in enumerate(num_hidden)])
  layers.append([
    tf.keras.layers.Dense(len(CLASS_NAMES),
                          kernel_regularizer=regularizer,
                          activation='softmax',
                          name='flower_prob')
  ])

  model = tf.keras.Sequential(layers, name='flower_classification')

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossEntropy(
                    from_logits=False),
                metrics=['accuracy'])
  print(model.summary())

  history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)
  training_plot(['loss', 'accuracy'], history)

  return model

In [ ]:
model = train_and_evaluate(lrate=0.0001, l2=0.001, num_hidden=[64, 16])